In [1]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import string
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from tensorflow.keras import optimizers
import pickle
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint




import collections
import jieba

# Read File

In [2]:
with open('TsinghuaBilingualCorpus/HK.txt',encoding="utf8") as f:
    output = f.read()

In [3]:
text = output.split('\n')

In [4]:
len(text)

1261558

In [5]:
alphabet = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

c_filter = []
for i in range(0,len(text),2):
    if len(text[i]) <=10 and (all(x not in alphabet for x in text[i])):
        c_filter.append(text[i])
        c_filter.append(text[i+1])
    else: 
        pass

e_filter = []
for i in range(1,len(c_filter),2):
    if (len(c_filter[i]) <=20) and (all(x in alphabet or x.isspace() for x in c_filter[i])):
        e_filter.append(c_filter[i-1])
        e_filter.append(c_filter[i])
    else: 
        pass

In [6]:
len(e_filter)

33070

In [7]:
ch_en_dict = {e_filter[i]: e_filter[i + 1] for i in range(0, len(e_filter), 2)}

In [8]:
#list of eng and ch sentences
eng = list(ch_en_dict.values())
ch = list(ch_en_dict.keys())

In [9]:
train = []
for i in range(len(eng)):
    tmp = [eng[i],ch[i]]
    train.append(tmp)

 # train, test = train_test_split(train, test_size=0.20, random_state=42)

In [10]:
len(eng)

13149

In [11]:
len(ch)

13149

In [12]:
english_words_counter = collections.Counter([word for sentence in eng for word in sentence.split()])
chinese_words_counter = collections.Counter([word for sentence in ch for word in list(jieba.cut(sentence,cut_all = False))])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Steambun\AppData\Local\Temp\jieba.cache
Loading model cost 0.519 seconds.
Prefix dict has been built successfully.


In [13]:
print('{} unique English words.'.format(len(english_words_counter)))
print('{} English words.'.format(len(eng)))
#print('10 Most common words in the English dataset:')
#print('"' + '" "'.join(list(zip(*eng.most_common(10)))[0]) + '"')
print()
print('{} unique Chinese characters.'.format(len(chinese_words_counter)))
print('{} Chinese characters.'.format(len(ch)))
#print('10 Most common words in the Chinese dataset:')
#print('"' + '" "'.join(list(zip(*ch.most_common(10)))[0]) + '"')

7185 unique English words.
13149 English words.

9509 unique Chinese characters.
13149 Chinese characters.


# Clean Data 

In [14]:
# Define a function to remove punctuation in our messages and lower case
def remove_punct(text):
    text = "".join([char for char in text if char not in string.punctuation])
    return text.lower()

for i in range(len(eng)):
    eng[i] = remove_punct(eng[i])
    
for i in range(len(ch)):
    ch[i] = remove_punct(ch[i])
    ch[i] = list(jieba.cut(ch[i],cut_all = False))

In [15]:
eng[0]

'skip to search box '

In [16]:
seg_list = list(jieba.cut("我来到北京清华大学", cut_all=False))
print("Default Mode: " + "/ ".join(seg_list))

Default Mode: 我/ 来到/ 北京/ 清华大学


# Tokenizer

In [17]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer

In [18]:
eng_tokenizer = tokenize(eng)
eng_vocab_size = len(eng_tokenizer.word_index) + 1

ch_tokenizer = tokenize(ch)
ch_vocab_size = len(ch_tokenizer.word_index) + 1

In [19]:
ch[0]

['跳至', '搜寻']

In [20]:
ch_tokenizer.texts_to_sequences(ch[0:10])

[[1619, 162],
 [1619, 2263],
 [2264],
 [79, 53],
 [76, 2],
 [19],
 [119, 1, 3648, 3649],
 [2263],
 [608, 541],
 [162]]

In [21]:
eng_tokenizer.texts_to_sequences(eng[0:10])

[[1309, 6, 82, 629],
 [1309, 6, 303, 942],
 [630],
 [117, 943],
 [13],
 [631, 54],
 [118, 1310],
 [303, 942],
 [712],
 [105, 82]]

In [22]:
# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
         # integer encode sequences
    seq = tokenizer.texts_to_sequences(lines)
         # pad sequences with 0 values
    seq = pad_sequences(seq, maxlen=length, padding='post')
    return seq

In [23]:
def max_length(lines):
    return max(len(line.split()) for line in lines)

In [24]:
max_eng = max_length(eng)
max_ch = 10

In [25]:
# one hot encode target sequence
def encode_output(sequences, vocab_size):
    ylist = list()
    for sequence in sequences:
        encoded = to_categorical(sequence, num_classes=vocab_size)
        ylist.append(encoded)
    y = np.array(ylist)
    y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
    return y

In [26]:
trainX, testX, trainY, testY = train_test_split(encode_sequences(eng_tokenizer,max_eng,eng),encode_sequences(ch_tokenizer,max_ch,ch), test_size=0.20, random_state=42)
trainY = encode_output(trainY, ch_vocab_size)
testY = encode_output(testY, ch_vocab_size)

In [27]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint
from nltk.translate.bleu_score import corpus_bleu

In [28]:


# define NMT model
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
    # Hyperparameters
    learning_rate = 0.005
    
    model = Sequential()
    model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
    model.add(LSTM(n_units))
    model.add(RepeatVector(tar_timesteps))
    model.add(LSTM(n_units, return_sequences=True))
    model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
    return model

# define model
model = define_model(eng_vocab_size, ch_vocab_size, max_eng, max_ch, 200)
model.compile(optimizer='adam', loss='categorical_crossentropy')
# summarize defined model
print(model.summary())
#plot_model(model, to_file='model.png', show_shapes=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 5, 200)            1119800   
_________________________________________________________________
lstm (LSTM)                  (None, 200)               320800    
_________________________________________________________________
repeat_vector (RepeatVector) (None, 10, 200)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 200)           320800    
_________________________________________________________________
time_distributed (TimeDistri (None, 10, 9501)          1909701   
Total params: 3,671,101
Trainable params: 3,671,101
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
#import tensorflow as tf
#with tf.device('/CPU:0'):
#    model.fit(X_train, y_train,epochs=1, batch_size=512, verbose=1)


In [46]:
# fit model
#filename = 'model.h5'
#checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(trainX, trainY, epochs=100, batch_size=32, validation_data=(testX, testY),verbose=1)

Epoch 1/100
329/329 [==============================] - 9s 26ms/step - loss: 0.1449 - val_loss: 2.7267
Epoch 2/100
329/329 [==============================] - 9s 26ms/step - loss: 0.1456 - val_loss: 2.7395
Epoch 3/100
329/329 [==============================] - 9s 26ms/step - loss: 0.1463 - val_loss: 2.7437
Epoch 4/100
329/329 [==============================] - 9s 27ms/step - loss: 0.1445 - val_loss: 2.7502
Epoch 5/100
329/329 [==============================] - 9s 26ms/step - loss: 0.1432 - val_loss: 2.7482
Epoch 6/100
329/329 [==============================] - 9s 27ms/step - loss: 0.1425 - val_loss: 2.7420
Epoch 7/100
329/329 [==============================] - 9s 26ms/step - loss: 0.1421 - val_loss: 2.7434
Epoch 8/100
329/329 [==============================] - 9s 26ms/step - loss: 0.1418 - val_loss: 2.7524
Epoch 9/100
329/329 [==============================] - 9s 27ms/step - loss: 0.1416 - val_loss: 2.7443
Epoch 10/100
329/329 [==============================] - 9s 26ms/step - loss: 0.142

In [31]:
# evaluate the skill of the model
def evaluate_model(model, tokenizer, sources, raw_dataset):
    actual, predicted = list(), list()
    for i, source in enumerate(sources):
        # translate encoded source text
        source = source.reshape((1, source.shape[0]))
        translation = predict_sequence(model, eng_tokenizer, source)
        raw_target, raw_src = raw_dataset[i]
        if i < 10:
            print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
        actual.append([raw_target.split()])
        predicted.append(translation.split())
    # calculate BLEU score
    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

In [37]:
# map an integer to a word
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [38]:
# generate target given source sequence
def predict_sequence(model, tokenizer, source):
    prediction = model.predict(source, verbose=0)[0]
    integers = [np.argmax(vector) for vector in prediction]
    target = list()
    for i in integers:
        word = word_for_id(i, tokenizer)
        if word is None:
            break
        target.append(word)
    return ' '.join(target)

In [39]:
# test on some training sequences
print('train')
#evaluate_model(model, eng_tokenizer, trainX, train)
# test on some test sequences
print('test')
#evaluate_model(model, ch_tokenizer, testX, test)

train
test


In [45]:
translate('all I want for christmas')

'注册 名单'

In [ ]:
#food,horse,concrete

In [41]:
def translate(text):
    text = remove_punct(text)
    text = encode_sequences(eng_tokenizer,max_eng,text.split())
    return predict_sequence(model,ch_tokenizer,text)

In [ ]:
train

In [ ]:
ch_tokenizer.index_word